In [ ]:
import http.server
import socketserver

PORT = 8000

ans_dict = {"A":0, "B":0, "C":0, "D":0, "R":0}

class Handler(http.server.SimpleHTTPRequestHandler):
            
    def do_GET(self):
        global ans_dict
        
        if not (self.path.endswith("/") or self.path.endswith("?")):
            self.send_response(404)
            self.end_headers()
            return
        
        R = "<input type=\"submit\" name=\"action\" value=\"R\" class=\"button\"/><br/>" if ans_dict['R'] == 0 else ""
        content = "<html>\
        <head><style>.button { background-color: #4CAF50; border: none; color: white;\
        padding: 16px 32px; text-align: center; display: inline-block; font-size: 36px; margin: 8px 4px; }\
        </style></head>\
        <body><form method='post' action=\"/\">\
        <input type=\"submit\" name=\"action\" value=\"A\" class=\"button\"/><br/\>\
        <input type=\"submit\" name=\"action\" value=\"B\" class=\"button\"/><br/>\
        <input type=\"submit\" name=\"action\" value=\"C\" class=\"button\"/><br/>\
        <input type=\"submit\" name=\"action\" value=\"D\" class=\"button\"/><br/>"
        content += R
        content += "</form></body></html>"
        content = content.encode("ascii")

        self.send_response(200)
        self.send_header('Content-type', 'text/html')
        self.send_header("Content-length", len(content))
        self.end_headers()
        self.wfile.write(content)
        return
            
    def do_POST(self):
        global ans_dict
        
        if not (self.path.endswith("/") or self.path.endswith("?")):
            self.send_response(404)
            self.end_headers()
            return
        
        content_len = int(self.headers.get('content-length', 0))
        post_body = self.rfile.read(content_len)
        ans = post_body.decode('ascii').split("=")[-1]
        if ans in ans_dict:
            ans_dict[ans] += 1
            #print(ans_dict)
        if ans[0] == 'R':
            
            if ans == "Restart":
                ans_dict = {"A":0, "B":0, "C":0, "D":0, "R":1}
            
            content = b"<html><head>\
            <script type=\"text/javascript\" src=\"https://www.gstatic.com/charts/loader.js\"></script>\
            <script type=\"text/javascript\">\
            google.charts.load('current', {packages: ['corechart', 'bar']});\
            google.charts.setOnLoadCallback(drawChart);\
            function drawChart() {\
            var data = google.visualization.arrayToDataTable([\
            ['Answer', 'Count', ],\
            ['A', "+str(ans_dict['A']).encode('ascii')+b"],\
            ['B', "+str(ans_dict['B']).encode('ascii')+b"],\
            ['C', "+str(ans_dict['C']).encode('ascii')+b"],\
            ['D', "+str(ans_dict['D']).encode('ascii')+b"]\
            ]);\
            var options = { title: \"Answers\", chartArea: {width: '50%', left: 30}};\
            var chart = new google.visualization.BarChart(document.getElementById(\"barchart_values\"));\
            chart.draw(data, options);\
            }\
            </script>\
            </head><body>\
            <div id=\"barchart_values\"></div>\
            <form method='post'><input type=\"submit\" name=\"action\" value=\"Refresh\" /></form>\
            <form method='post'><input type=\"submit\" name=\"action\" value=\"Restart\" /></form>\
            </body></html>"
            
            self.send_response(200)
            self.send_header('Content-type', 'text/html')
            self.send_header("Content-length", len(content))
            self.end_headers()
            self.wfile.write(content)
            return
        
        #
        content = b"<html>\
        <head><style>.button { background-color: #4CAF50; border: none; color: white;\
        padding: 16px 32px; text-align: center; display: inline-block; font-size: 36px; margin: 8px 4px; }\
        </style></head>\
        <body><form id='f' method='get' action=\"/\">\
        <button class=\"button\" onclick=\"document.getElementById(\"f\").submit()\">Back</button></form></body></html>"
        
        self.send_response(200)
        self.send_header('Content-type', 'text/html')
        self.send_header("Content-length", len(content))
        self.end_headers()
        self.wfile.write(content)
        return
        
with socketserver.ThreadingTCPServer(("", PORT), Handler) as httpd:
    print("serving at port", PORT)
    httpd.serve_forever()